<a href="https://colab.research.google.com/github/T2718/AI_Make_Image/blob/main/CivitAI5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# --- モード選択 ---
print("=== モード選択 ===")
print("1: Hugging Faceで画像生成")
print("2: CivitAI（モデルDL / safetensors変換）")
print("3: トークン数を数える")
print("4: 出力フォルダを削除")
mode = input("番号を入力してください: ").strip()

# --- 必要なパラメータを先に全て入力 ---
hf_params = {}
civitai_params = {}

model_list_H = ['stablediffusionapi/eleet-model',
              'stablediffusionapi/brav6',
              'Vsukiyaki/ShiratakiMix',
              'stabilityai/stable-diffusion-xl-base-1.0'
              ]

model_list_C = ["https://civitai.com/api/download/models/28100?type=Model&format=SafeTensor&size=full&fp=fp16",
                "https://civitai.com/api/download/models/11812?type=Model&format=SafeTensor&size=full&fp=fp16"
              ]



if mode == "1":
  hf_params["model_id"] = input("HuggingFaceのモデルIDを入力（例: runwayml/stable-diffusion-v1-5）: ").strip()
  hf_params["submode"] = input("生成モードを選択（1: text2img, 2: img2img）: ").strip()
  hf_params["prompt"] = input("プロンプトを入力: ").strip()
  hf_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  hf_params["num"] = int(input("生成する画像の枚数 : "))
  hf_params["steps"] = int(input("ステップ数（例: 70）: "))
  hf_params["guidance"] = float(input("guidance scale（例: 7.5）: "))
  if hf_params["submode"] == "2":
    hf_params["strength"] = float(input("strength (例: 0.75) : "))
    hf_params["image_path"] = input("初期画像のパスを入力してください（img2img用）: ")

elif mode == "2":
  print("--- CivitAI モデルの状況は？ ---")
  print("1: まだ何もしていない（DLも変換もしていない）")
  print("2: safetensorsファイルのみアップロード済み（変換が必要）")
  print("3: 変換済みでそのまま使える")
  civitai_params["status"] = input("番号を入力してください: ").strip()
  #civitai_params["model_name"] = input("保存先モデル名（例: models）: ").strip()
  if civitai_params["status"] == "1":
    civitai_params["url"] = input("CivitAIのモデル/LoRA ダウンロードURL: ").strip()
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
    civitai_params["safetensorsName"] = input("ファイル名(downloaded_model.safetensors) : ")
    if civitai_params["safetensorsName"] == "":
      civitai_params["safetensorsName"] = "downloaded_model.safetensors"
  elif civitai_params["status"] == "2":
    civitai_params["safefile"] = input("アップロード済みの .safetensors ファイル名(downloaded_model.safetensors): ").strip()
    if civitai_params["safefile"] == "":
      civitai_params["safefile"] = "downloaded_model.safetensors"
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
  civitai_params["prompt"] = input("プロンプトを入力してください: ")
  civitai_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  civitai_params["num"] = int(input("生成する画像の枚数 : "))
  civitai_params["steps"] = int(input("ステップ数（例: 70）: "))
  civitai_params["lora_path"] = input("LoRAを使用する場合はpathを(使わない場合は無入力)")
  if civitai_params["lora_path"] == "":
    civitai_params["lora_path"] = None

elif mode == "3":
  token_prompt = input("プロンプトを入力してください（トークン数確認）: ").strip()

elif mode == "4":
  confirm_delete = input("本当にoutputフォルダを削除しますか？ (y/n): ").strip().lower()

# --- 必要なinstallはここで実行（全てのinput終了後） ---
print("インストールを開始します...")
!pip install diffusers transformers accelerate safetensors

# --- 各モード実行 ---
if mode == "1":
  from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
  from PIL import Image
  import torch
  import os
  torch_dtype = torch.float16
  if hf_params["submode"] == "1":
    pipe = StableDiffusionPipeline.from_pretrained(
      hf_params["model_id"],
      torch_dtype=torch.float16,
      safety_checker=None  # Safe checkを無効に
    )
    pipe = pipe.to("cuda")  # GPUが必要です
  else:
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        hf_params["model_id"],
        torch_dtype=torch.float16,
        safety_checker=None
    )
    pipe = pipe.to("cuda")
  for k in range(hf_params["num"]):
    if hf_params["submode"] == "1":

      image = pipe(hf_params["prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   guidance_scale=hf_params["guidance"],
                   added_cond_kwargs={}).images[0]
    else:
      init_image = Image.open(hf_params["image_path"]).convert("RGB").resize((512, 512))
      image = pipe(prompt=hf_params["prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   image=init_image,
                   strength=hf_params["strength"],
                   guidance_scale=hf_params["guidance"]).images[0]

    os.makedirs("output", exist_ok=True)
    filename = f"output/huggingface_result"+str(k)+".png"
    image.save(filename)
  print("画像を保存しました")

elif mode == "2":
  import os
  import torch
  import re
  from diffusers import StableDiffusionPipeline

  base_dir = "/content/models"
  os.makedirs(base_dir, exist_ok=True)

  if civitai_params["status"] == "1":
    regex = r'([0-9]{5})'
    if bool(re.match(regex, civitai_params['url'])):
      civitai_params['url'] = 'https://civitai.com/api/download/models/'+civitai_params['url']
    elif civitai_params['url'] == '1':
      civitai_params['url'] = "https://civitai.com/api/download/models/28100?type=Model&format=SafeTensor&size=full&fp=fp16"
    elif civitai_params['url'] == '2':
      civitai_params['url'] = "https://civitai.com/api/download/models/11812?type=Model&format=SafeTensor&size=full&fp=fp16"


    download_path = os.path.join(base_dir, civitai_params["safetensorsName"])

    model_url = civitai_params["url"]
    model_name = civitai_params["safetensorsName"]

    print(f"Downloading Checkpoint model to {download_path}...")
    !wget -O {download_path} "{model_url}"
    print(f"Download of {model_name} complete!")

    # Checkpointモデルのパスを記録
    checkpoint_path = download_path
    lora_path = civitai_params["lora_path"] # LoRAはダウンロードしないのでNone


    if civitai_params["need_yaml"]:
      !wget -O /content/models/v1-inference.yaml \
https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
    !mkdir /content/converted
    !wget -O convert_original_stable_diffusion_to_diffusers.py \
    https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
    os.makedirs(base_dir, exist_ok=True)
    !python convert_original_stable_diffusion_to_diffusers.py \
      --checkpoint_path ./models/downloaded_model.safetensors \
      --original_config_file ./models/v1-inference.yaml \
      --dump_path ./converted/diffusers_model \
      --from_safetensors

  elif civitai_params["status"] == "2":
    if civitai_params["need_yaml"]:
      !wget -O /content/models/v1-inference.yaml \
https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
      #!wget -q https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-inference.yaml
    !mkdir /content/converted
    !wget -O convert_original_stable_diffusion_to_diffusers.py \
    https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
    os.makedirs(base_dir, exist_ok=True)
    !python convert_original_stable_diffusion_to_diffusers.py \
      --checkpoint_path ./models/downloaded_model.safetensors \
      --original_config_file ./models/v1-inference.yaml \
      --dump_path ./converted/diffusers_model \
      --from_safetensors

  #pipe = StableDiffusionPipeline.from_pretrained(base_dir, torch_dtype=torch.float16).to("cuda")
  pipe = StableDiffusionPipeline.from_pretrained(
      "/content/converted/diffusers_model",
      torch_dtype=torch.float16,
      use_safetensors=False,
      safety_checker=None
    ).to("cuda")
  # パイプラインを作成した直後に以下を追加します
  if lora_path:
    pipe.load_lora_weights(lora_path, use_safetensors=True)

  #pipe.enable_xformers_memory_efficient_attention()
  for k in range(civitai_params["num"]):
    image = pipe(prompt=civitai_params["prompt"], negative_prompt=civitai_params["negative_prompt"], num_inference_steps=civitai_params["steps"]).images[0]
    os.makedirs("output/"+civitai_params["prompt"][0:10], exist_ok=True)
    image.save("output/"+civitai_params["prompt"][0:10]+"/"+str(k)+".png")
  print("画像を保存しました: output/"+civitai_params["prompt"][0:10]+".png")

elif mode == "3":
  from transformers import CLIPTokenizer
  tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
  tokens = tokenizer(token_prompt)["input_ids"]
  print(f"トークン数: {len(tokens)}")

elif mode == "4":
  import shutil
  if confirm_delete == "y" and os.path.exists("output"):
    shutil.rmtree("output")
    print("outputフォルダを削除しました")
  else:
    print("キャンセルされました")


=== モード選択 ===
1: Hugging Faceで画像生成
2: CivitAI（モデルDL / safetensors変換）
3: トークン数を数える
4: 出力フォルダを削除
番号を入力してください: 2
--- CivitAI モデルの状況は？ ---
1: まだ何もしていない（DLも変換もしていない）
2: safetensorsファイルのみアップロード済み（変換が必要）
3: 変換済みでそのまま使える
番号を入力してください: 3
プロンプトを入力してください: anime
ネガティブプロンプトを入力 : real
生成する画像の枚数 : 10
ステップ数（例: 70）: 100
LoRAを使用する場合はpathを(使わない場合は無入力)
インストールを開始します...


RuntimeError: Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
Failed to import transformers.models.clip.image_processing_clip because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [4]:
import os
base = '/content/models'
Name = 'downloaded_model.safetensors'
download_path = os.path.join(base, Name)

model_url = input('url')
!mkdir {base}
print(f"Downloading Checkpoint model to {download_path}...")
!wget -O {download_path} "{model_url}"
print(f"Download complete!")

urlhttps://civitai.com/api/download/models/1410435?type=Model&format=SafeTensor&size=pruned&fp=fp16
mkdir: cannot create directory ‘/content/models’: File exists
--2025-05-21 23:41:24--  https://civitai.com/api/download/models/1410435?type=Model&format=SafeTensor&size=pruned&fp=fp16
Resolving civitai.com (civitai.com)... 172.67.12.143, 104.22.18.237, 104.22.19.237, ...
Connecting to civitai.com (civitai.com)|172.67.12.143|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://civitai-delivery-worker-prod.5ac0637cfd0766c97916cefa3764fbdf.r2.cloudflarestorage.com/model/31176/waiNsfwIllustrious11.Nmu2.safetensors?X-Amz-Expires=86400&response-content-disposition=attachment%3B%20filename%3D%22waiNSFWIllustrious_v110.safetensors%22&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=e01358d793ad6966166af8b3064953ad/20250521/us-east-1/s3/aws4_request&X-Amz-Date=20250521T234125Z&X-Amz-SignedHeaders=host&X-Amz-Signature=dc80b1d81c6862cb9ecb62131a45116b

In [3]:
from diffusers import StableDiffusionPipeline
import torch

# safetensorsファイルのパス（例: Colabでアップロードしたファイル名）
model_path = "/content/models/downloaded_model.safetensors"  # ←アップロード済みのファイル名に置き換えてください

# モデル読み込み
pipe = StableDiffusionPipeline.from_single_file(
  model_path,
  torch_dtype=torch.float16,
  use_safetensors=True
)
pipe.to("cuda")

# プロンプトを指定して画像生成
prompt = "masterpiece, best quality, 1girl"

# added_cond_kwargsを明示的に設定
# エラーメッセージとdiffusersのコードから、text_embedsとtime_idsが必要になる可能性がある
added_cond_kwargs = {
    "text_embeds": torch.randn(1, 1280).half().cuda(), # ダミーのtext_embeds
    "time_ids": torch.randn(1, 6).half().cuda() # ダミーのtime_ids
}

image = pipe(prompt, added_cond_kwargs=added_cond_kwargs).images[0]

# 保存
image.save("output.png")
image.show()  # Colabなら画像表示も

RuntimeError: Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
Failed to import transformers.models.clip.image_processing_clip because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [1]:
# @title 1. 環境構築とモデルダウンロードの準備
# 必要なライブラリのインストール
print("Installing required libraries...")
!pip install -q diffusers transformers accelerate safetensors xformers
print("Libraries installed.")

# ダウンロード先のディレクトリ設定
import os
base_dir = "/content/models"
os.makedirs(base_dir, exist_ok=True)

# -----------------------------------------------------------------------------
# ここに提供されたCheckpointモデルのURLを直接設定
# https://civitai.com/api/download/models/1410435?type=Model&format=SafeTensor&size=pruned&fp=fp16
checkpoint_url = "https://civitai.com/api/download/models/1410435?type=Model&format=SafeTensor&size=pruned&fp=fp16"
checkpoint_filename = "civitai_sdxl_pruned_fp16.safetensors" # 分かりやすいファイル名を指定
checkpoint_path = os.path.join(base_dir, checkpoint_filename)
# -----------------------------------------------------------------------------


# @title 2. Checkpointモデルのダウンロード
print(f"\n--- Downloading Checkpoint Model ---")
print(f"URL: {checkpoint_url}")
print(f"Saving to: {checkpoint_path}")

try:
    # -q: quiet (静かに), -O: output document to file (指定したファイル名で保存)
    !wget -q -O {checkpoint_path} "{checkpoint_url}"

    # ダウンロードの成功を確認
    if not os.path.exists(checkpoint_path) or os.path.getsize(checkpoint_path) == 0:
        raise Exception("ダウンロードに失敗しました。ファイルが正しく保存されなかったか、サイズが0です。URLを確認してください。")
    print(f"Download of '{checkpoint_filename}' complete! Size: {os.path.getsize(checkpoint_path) / (1024*1024):.2f} MB")

except Exception as e:
    print(f"Checkpointモデルのダウンロード中にエラーが発生しました: {e}")
    print("処理を中断します。URLが正しいか、Civitaiのダウンロード制限などに引っかかっていないか確認してください。")
    import sys
    sys.exit(1) # Colabの実行を停止


# @title 3. Stable Diffusion パイプラインのロード
import torch
from diffusers import AutoPipelineForText2Image

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"\n--- Loading Stable Diffusion Pipeline ---")
print(f"Loading model from: {checkpoint_path}")

try:
    # from_single_fileを使用して.safetensors形式のモデルをロード
    pipeline = AutoPipelineForText2Image.from_single_file(
        checkpoint_path,
        torch_dtype=torch.float16, # GPUメモリ節約のためfloat16推奨
        safety_checker=None       # NSFW制限を解除
    )
    pipeline.to(device) # モデルをGPUに移動

    # XFormersの利用 (高速化とメモリ効率化)
    if torch.cuda.is_available():
        pipeline.enable_xformers_memory_efficient_attention()
        print("XFormers memory-efficient attention enabled.")

    print("Checkpoint model loaded successfully into pipeline.")

except Exception as e:
    print(f"Checkpointモデルのロード中にエラーが発生しました: {e}")
    print("モデルファイルが破損しているか、diffusersでサポートされていない形式かもしれません。")
    print("処理を中断します。")
    import sys
    sys.exit(1)


# @title 4. 画像生成の実行
print("\n--- Image Generation Settings ---")
prompt = input("Enter your prompt for image generation: ")
negative_prompt = input("Enter negative prompt (e.g., low quality, blurry, distorted): ")

try:
    num_inference_steps = int(input("Enter number of inference steps (default: 30): ") or "30")
except ValueError:
    num_inference_steps = 30
    print("Invalid input for steps, setting to 30.")

try:
    guidance_scale = float(input("Enter Guidance Scale (default: 7.5): ") or "7.5")
except ValueError:
    guidance_scale = 7.5
    print("Invalid input for guidance scale, setting to 7.5.")

seed_input = input("Enter a seed for reproducibility (leave empty for random): ")
seed = int(seed_input) if seed_input.isdigit() else None

generator = None
if seed is not None:
    generator = torch.Generator(device=device).manual_seed(seed)
    print(f"Using manual seed: {seed}")
else:
    print("Using random seed.")

print("\nGenerating image...")
try:
    image = pipeline(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator, # シード値を適用
    ).images[0]

    # 生成された画像を保存または表示
    output_filename = "generated_image.png"
    image.save(output_filename)
    print(f"Image generated and saved as '{output_filename}'.")

    # Colabで画像を表示する場合
    from IPython.display import Image, display
    display(Image(filename=output_filename))

except Exception as e:
    print(f"An error occurred during image generation: {e}")
    print("Please check your prompt, parameters, or GPU memory usage.")

finally:
    # @title 5. クリーンアップ (オプション)
    clean_up = input("\nDo you want to delete the model files after this session? (y/n): ").lower()
    if clean_up == 'y':
        if checkpoint_path and os.path.exists(checkpoint_path):
            os.remove(checkpoint_path)
            print(f"Checkpoint model '{checkpoint_path}' deleted.")
        print("Cleanup complete. Files are removed from Colab's temporary storage.")
    else:
        print("Model files will not be deleted from Colab's temporary storage.")

Installing required libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━

RuntimeError: Failed to import diffusers.pipelines.auto_pipeline because of the following error (look up to see its traceback):
Failed to import transformers.models.auto.image_processing_auto because of the following error (look up to see its traceback):
operator torchvision::nms does not exist